(cont:mm:intro)=
# Memory Management

- Introduction: Fundamental jobs of the memory manager:
    - Abstracting a useful memory model.
    - Managing the physical memory on the system.
    - Providing and managing a private virtual address space for each user.

- The memory manager is responsible for:
    - 1.) Managing all of the RAM or physical memory on the whole system. 
    - 2.) Creating and managing virtual memory for each process. 
    - 3.) Transitioning the processor(s) between physical memory mode and virtual memory mode as the kernel boots and initializes.

(cont:mm:overview)=
## Overview of MM
  ### Simple physical memory management.
The simplest memory management model is to just use and share the physical address space without any translation. This implies processes sharing that physical address space are randomly scattered around in it.  Each process also must deal with different addresses and there is no protection between those processes.  While this is very primitive, physical memory management is still used in simple embedded systems.
  
  ### Sharing one physical address space.
Early in the development of computer hardware and operating systems physical memory management was used.  Users were typically limited to trusted colleges and few users shared the computer at the same time.  Over time the quantity of memory expanded allowing more users to run programs on the computer at the same time.  In addition to a lack of protection sharing a single physical address space requires every program to deal with different physical addresses since its not know exactly where they will be loaded and run.  Each time a program is run its likely to loaded and run at a different address.  Dealing with this imposes a significant burden on basic programming.  To eliminate this burden operating system designers developed a relocation technique that corrected all the physical addresses when a program was loaded as a specific address.     
  ### Relocating programs.
When a program is compiled and linked into object code the address at which the program will run must be known so that branch destinations can be established.  When the program is loaded into memory it must be loaded at the correct address or the program will branch to the wrong locations.  Since more than one program can run at the same time there is no guarantee that the program will be loaded and run at the address it was linked to run.  For this reason programs are usually linked as relocatable.  Relocatable programs have a table of all addresses within the program image that must be changed if it is loaded at a different address than it was linked before it can be run.   This is how more that one program can be run at the same time in a system that only supports physical memory management.
  ## Virtual memory management: talk about all the advances
As discussed above physical memory management model has significant drawbacks, lack of protection and mandatory relocation being the most significant.  For this reason memory management subsystem has evolved over time give each process its own private address space known as a virtual address space.  The virtual address space for each process looks identical, that is starting at zero going up to some limit and provides protection from other virtual address spaces so that one process can not corrupt any other processes.
  ### Segmentation.
In the simplest form of segmentation the hardware provides 2 registers that are loaded each time a process acquire the CPU, a base register and a limit register.  For a given process the base register contains the physical address that the program was loaded at and the limit register contains the size of the program that was loaded into memory.  Every process has a virtual address space starting at zero and a size determined by the actual program size that the process is running.  The process specific base and limit registers which are loaded every time a process acquires the CPU establishes the bounds of the virtual address space for every process.  For each and every memory reference the hardware adds the virtual address to the base register to determine a physical address and insures that the physical address is between the base register and base register plus the limit register.  If it is outside those bounds the program is terminated with an illegal virtual memory reference error.  Segmentation solves both the lack of protection and the mandatory relocation requirements of physical addressing.  Segmentation has little or no performance overhead because the hardware performs the virtual to physical translation or the addition of the virtual address and the base register to determine every physical address.
  #### Single segments per address space
So far we discussed a segmentation implementation that provides one base register and one limit register in hardware and one of each of those process specific values that gets loaded into those registers when context switching to a given process.  Since there is only one of each register, the entire process virtual address space must be physically contiguous and all text, data and stack must be within that single memory region.  While this is a huge improvement over a physical memory model it limits the size of the virtual address space to being static and not expandable.  There is no way to dynamically increase the size of the text, data or stack regions of a process at run-time, everything must be allocated in advance.  This requires allocating physical memory that might never be used.
  #### Multiple segments per address space.
As mentioned earlier a single base and limit segment register implies that an entire process virtual address space is a one physically contiguous region of physical memory mapped into one virtually contiguous virtual region of virtual memory.  This means that the text, data and stack regions must be packed tightly together in both physical and virtual memory unless we are willing to waste both physical and virtual memory.  Also, with only one segment register its not possible to offer different types of protections for the various regions of the virtual address space.  In other words all of virtual memory must be readable, writable and executable since data must be both readable and writable and text must be executable.  It would be nice to prevent data regions from being executable and text regions from being readable and writable for security and debug optimizations.

This can be achieved by the hardware implementing multiple segment and limit registers with only specified permissions for text, data and stack regions and having the operating system use those registers when context switching to a process.  When mapping the text into a virtual region the operating system can specify an execute only region that does not have to be adjacent to other non-executable regions.  When mapping data into virtual memory the operating system can specify read/write only thereby preventing execution of data regions.  Finally the stack can also be non-executable but also the operating system can move the virtual memory stack region away from any other region making it easier to debug common programming problems like stack overflows.  Finally multiple segment registers eliminates the necessity for the text, data and stack regions to be physically contiguous.  This allows a program to be split up into multiple smaller regions both physically and virtually making it much easier to hold more programs in physical memory at the same time.
  #### Private versus Global segments.
Every process virtual address space consists of 2 types of regions, Private and Global.  The private regions for a process are the text, data and stack regions for the program that process is running.  The global regions include the operating system that is and must be mapped into every process address space.  As we discussed earlier in this course the operating system consists of all the software that executes on behalf of the currently running program as well as basic system overhead that runs on behalf of the system.  This includes all the system calls the operating system supports
  #### Compaction.
  #### Limitations of segmentation.
  ### Paging: - now it could be unlimited... 
        - Virtual pages and page frames.
        - Virtual address spaces larger than RAM.
        - Page tables brief (more later)
    - Quick overview of what memory manager needs to do
        - compaction 
        - bringing things in, demand paging
            swapping, paging
        - deciding what to kick out - reclamation
        - anonymous memory
        - file backed and caching storage
        - example: look at elf, parts of the program...

(cont:mm:reclaim)=
## Memory reclaiming algorithms.
- Global page reclaiming
   - FIFO, LRU, NRU...
- Local page reclaiming
   - Reclaiming within a specific address space
- Working set page reclaiming

(cont:mm:ds)=
## Data structures
We need to keep track of ... what is free, what is used... 
- Free lists
   - coalescing free memory.
- In-use page lists
   - active and inactive lists for reclaiming.
   - Buffer Cache 
       - Using RAM to cache file system data & metadata
           - Global page hash
           - Per file page hash

(cont:mm:pf)=
## Page Faults
- File backed page faults
- Anonymous page faults
- Copy On Write page faults

(cont:mm:HW)=
## Memory management hardware
   - Translations
       - Page Table entries
       - Page tables
           - Multiple level page tables
           - Page table entries
           - Inverted page tables
           - Variable page sizes
   - TLB - Transpation Lookaside Buffers and entries
       - Global TLB entries
       - Local TLB entries
       - Tagging TLB entries
       

(cont:mm:dynamics)=
## Paging Dynamics
- Boot-time
- Consuming physical memory
   - Kernel page wiring
   - Page faults
   - Buffer Cache
- Freeing physical memory
   - Process exiting
   - un-mapping
   - Page reclaiming
       - Paging
       - Swapping
       - Page cache purging
- System shutdown

(cont:mm:linux)=
## Memory management in the real world

Linux specific memory management - examples from a real system


Memory Management Introduction

The memory management subsystem has 3 basic responsibilities; 
1. Providing an abstraction so that every user appears to have total and unique access to all of the memory on the system.  
2. Managing all of the physical memory on the system.  
3. Providing and managing a unique private virtual address space for every process currently running on the system. 

Very basic systems do not abstract memory at all.  Instead, programs are loaded directly into physical memory
and all the addresses presented to those programs are physical addresses.  This means that programs running in
a physical memory model must be relocatable because its not known what address they will be loaded and run at.
Being relocatable means that all addresses are either relative or they can be fixed up whenever the program is loaded and/or
moved to a different physical address.  

No memory abstraction also implies that there is no protection between multiple programs loaded into memory at the same time.
If a program takes a wild branch or writes to a random address it will corrupt other programs or even the entire system.

## Notes

Refer [here](https://jappavoo.github.io/UndertheCovers/textbook/unix/shellintro.html#arithmetic-expansion) for doing binary math in shell 